# Subtitle Dictionary Project

## Preparation

#### Imports

In [1]:
import numpy as np
import pandas as pd
import json
import sys
import srt
import string
import re

#### Configuring Environment

In [2]:
# Numpy / Pandas display options
np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 100)

# Enable multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

#### Setting Vars

In [3]:
# movie_title = "./data/"+"Spellbound 1945"
movie_title = "./data/"+"The.Lord.of.The.Rings.The.Two.Towers.2002"
movie_srt               = movie_title + ".srt"
movie_title_user_action = movie_title + '_user_action.csv'
# movie_title_user_action = movie_title + '_with_.csv'

user_name = "charlie"
user_ignore_file = "./data/w2ignore-" + user_name + ".csv"

w2ignore_csv = "./data/w2ignore.csv"

#### Words to Ignore

In [4]:
# df_w2i = pd.read_csv(w2ignore_csv).rename(columns = {"Words to Ignore": "w2ignore"})

In [5]:
# w2ignore = set(df_w2i.w2ignore)

## Parsing Subtitle

In [6]:
def read_st(st_file) -> str:
    
    with open(st_file, 'r') as f:
        st = f.read()
    
    st = st.replace('\n', ' \n')
    return st

In [7]:
def parse_st(st : str) -> str:
    
    subtitle_generator = srt.parse(st)    
    st = ''
    for l in subtitle_generator:
        st += l.content
        
    return st

In [8]:
def clean_st(st : str) -> str:

    st = st.replace('\n', ' ')
    st = st.replace('</i><i>', ' ')
    st = st.replace('</i>', ' ')
    st = st.replace('<i>', ' ')
    st = st.replace(string.punctuation, ' ')   # working ?
    st = st.replace('.', ' ')
    st = st.replace('\\\'s', ' ')
    st = st.replace('s\'', ' ')

    replace = ' '
    pattern = '[^a-zA-Z0-9 - ]'
    st = re.sub(pattern, replace, st) 

    pattern = '[\s+ \\\' \? 0-9 "]'
    st = re.sub(pattern, replace, st)     

    return st

In [9]:
def unique_st(st: str) -> list:
    st_list = st.lower().split(' ')
    st_list = sorted(list(set(st_list)))
    if st_list[0].strip() == '':
        st_list.pop(0)
#     st_list = sorted ( st_list )
    return st_list

In [10]:
def head_tail_st(st_list : list, count : int = 5):
    disp = st_list[:count] + ['...'] + st_list[-count:]
    return disp

def wordin_st(st: list, word : str):
    return st.index(word) if word in st else -1

## Word to Ignore

In [11]:
def df_w2ignore(w2ignore_csv : str):
    df_w2ignore = pd.read_csv(w2ignore_csv)
    return df_w2ignore

def w2ig_get_list(w2ignore_csv):
    df = df_w2ignore(w2ignore_csv)
    col = list(df.columns)[0]
    w2ignore_list = list(df[col])
    return w2ignore_list

# def w2ig_get_list(df_w2ignore):
#     col = list(df_w2ignore.columns)[0]
#     w2ignore_list = list(df_w2ignore[col])
#     return w2ignore_list

def w2ig_remove(st, w2ig):
    st = sorted( list( set(st) - set(w2ig) ))
    return st
    
def w2ig_process(st, w2ignore_csv):
    w2ig_list = w2ig_get_list(w2ignore_csv)
    return w2ig_remove( st , w2ig_list )

# def w2ig_process(st, w2ignore_csv):
#     df = df_w2ignore(w2ignore_csv)
#     w2ig_list = w2ig_get_list(df)
#     return w2ig_remove( st , w2ig_list )

## Save Subtitle to CSV

In [12]:
def srt_to_csv(st, movie_csv):     # st = subtitle (list),  movie_csv = filename of movie.csv
    srt_words = 'Subtitle Words'
    cols = [ srt_words , 'Seach' , 'Ignore']
    df = pd.DataFrame(columns = cols)
    df[srt_words] = st
    df.to_csv(movie_csv, index = False)

In [13]:
def save_srt(st, movie_csv):     # st = subtitle (list),  movie_csv = filename of movie.csv
    srt_words = 'Subtitle Words'
    cols = [ srt_words , 'Search' , 'Ignore']
    df = pd.DataFrame(columns = cols)
    df[srt_words] = st
    df.to_csv(movie_csv, index = False)
    return df

## Procesing Subtitle

#### Get list

In [14]:
movie_srt

'./data/The.Lord.of.The.Rings.The.Two.Towers.2002.srt'

In [15]:
## Start Here ##

st = read_st(movie_srt)
st = parse_st(st)
st = clean_st(st)
st = unique_st(st)

In [ ]:
st

#### Remove Ignore

In [17]:
len(st)
st = w2ig_process(st, w2ignore_csv)
len(st)

1539

1535

In [18]:
dfX = save_srt(st, movie_title_user_action)
# Think no need to dfX

## User Action

User should now open CSV file and select words:<br>
- to search dic<br>
- to ignore

In [19]:
print(f'File to open {movie_title_user_action}')

File to open ./data/The.Lord.of.The.Rings.The.Two.Towers.2002_user_action.csv


## Update w2ignore

In [20]:
user_ignore_file

'./data/w2ignore-charlie.csv'

In [21]:
dfStage2 = pd.read_csv(movie_title_user_action)
dfUserIgnoreFile = pd.read_csv(user_ignore_file,squeeze=True)

In [22]:
dfStage2
dfUserIgnoreFile

,Subtitle Words,Search,Ignore
0,able,NaN,NaN
1,about,NaN,NaN
2,above,NaN,NaN
3,accept,NaN,NaN
4,account,NaN,NaN
...,...,...,...
1530,yesterday,NaN,NaN
1531,yet,NaN,NaN
1532,young,NaN,NaN
1533,your,NaN,NaN


0         last
1    yesterday
2         able
Name: Ignore, dtype: object

In [23]:
dfStage2["Ignore"].notna()

0       False
1       False
2       False
3       False
4       False
        ...  
1530    False
1531    False
1532    False
1533    False
1534    False
Name: Ignore, Length: 1535, dtype: bool

In [24]:
dfUserIgnoreFile

0         last
1    yesterday
2         able
Name: Ignore, dtype: object

In [25]:
type(dfUserIgnoreFile)

pandas.core.series.Series

In [26]:
# new_ignore_word = sorted(list(set(  dfStage2[dfStage2["Ignore"].notna()]["Subtitle Words"] 
#     .append(dfUserIgnoreFile[dfUserIgnoreFile.notna()])   )))
# new_ignore_word

In [27]:
dfUserIgnoreFilenew = ( dfStage2[dfStage2["Ignore"].notna()]["Subtitle Words"] 
    .append(dfUserIgnoreFile[dfUserIgnoreFile.notna()]))
type(dfUserIgnoreFilenew)

pandas.core.series.Series

In [28]:
dfUserIgnoreFilenew

0         last
1    yesterday
2         able
dtype: object

In [29]:
dfUserIgnoreFilenew.drop_duplicates(inplace=True)

In [30]:
dfUserIgnoreFilenew.to_csv("./data/temp.csv", index=False)

In [31]:
dfUserIgnoreFile

0         last
1    yesterday
2         able
Name: Ignore, dtype: object

In [32]:
dfUserIgnoreFilenew.to_csv("./data/temp.csv")

In [33]:
dfUserIgnoreFile['Ignore'] = new_ignore_word

NameError: name 'new_ignore_word' is not defined

In [ ]:
new_ignore_word

In [ ]:
df[df["Ignore"].notna()]

## Saving Subtitle to CSV

In [ ]:
srt_to_csv(st, movie_csv)

In [ ]:
# check file time of creation & head the output file

!dir ".\data\Spellbound 1945.csv"
!head  ".\data\Spellbound 1945.csv"

## Update w2ignore with new selected ignore words

In [ ]:
df

In [ ]:
A = 10
B = 11
C = 12

In [ ]:
A
B
C

In [ ]:
print(A)
print(B)
print(A)


# Checkpoint here - Last done till above

# Check point - Daddy worked up to here (above).  Below is API calls (codes as we have done previously.  Above are the added ignore file, write process output to csv

## Calling Dictionary API

In [ ]:
import urllib.request
import json

serviceinput = 'input1'
serviceoutput = 'output1'

def DicReq():
    
    url = 'https://api.dictionaryapi.dev/api/v2/entries/en/word'
    url = 'https://www.dictionaryapi.com/api/v3/references/learners/json/apple'
    url = 'https://www.dictionaryapi.com/api/v3/references/learners/json/apple?key=8ed98e54-182a-4fa3-b26a-8e22aae9caa9'
    api_key = '8ed98e54-182a-4fa3-b26a-8e22aae9caa9'
    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
    
    req = urllib.request.Request(url, headers=headers)

    try:
        response = urllib.request.urlopen(req)
        result = response.read()
        return result

    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))
        print(error.info())
        print(json.loads(error.read().decode("utf8", 'ignore')))        

In [ ]:
resp = DicReq()

In [ ]:
json.loads(resp)

## Sample Output for Testing

In [ ]:
false = False
true = True

In [ ]:
dicX = [{"meta":{"id":"apple","uuid":"e3bcad1b-e6cc-4e94-98d2-0efa435d84ed","src":"learners","section":"alpha","target":{"tuuid":"44e59f49-db3f-4db0-9639-788033a87df7","tsrc":"collegiate"},"highlight":"yes","stems":["apple","apples","compare apples to apples\\/oranges","compare apples to apples","compare apples to oranges","the apple of someone\\u0027s eye","the apple of someones eye","upset the apple cart","bad apple","rotten apple"],"app-shortdef":{"hw":"apple","fl":"noun","def":["{bc} a round fruit with red, yellow, or green skin and firm white flesh"]},"offensive":false},"hwi":{"hw":"ap*ple","prs":[{"ipa":"\\u02c8\\u00e6p\\u0259l","sound":{"audio":"apple001"}}]},"fl":"noun","ins":[{"il":"plural","if":"ap*ples"}],"def":[{"sseq":[[["sense",{"dt":[["text","{bc}a round fruit with red, yellow, or green skin and firm white flesh "],["wsgram","count"],["vis",[{"t":"crisp juicy {it}apples{\\/it}"},{"t":"a bad\\/rotten {it}apple{\\/it} [=an apple that has rotted and cannot be eaten]"}]],["wsgram","noncount"],["vis",[{"t":"a piece of {it}apple{\\/it}"}]],["uns",[[["text","often used before another noun "],["vis",[{"t":"{it}apple{\\/it} pie"},{"t":"{it}apple{\\/it} juice"},{"t":"{it}apple{\\/it} trees"}]]]]],["text"," "],["snote",[["t","In figurative use, a {phrase}bad apple{\\/phrase} or {phrase}rotten apple{\\/phrase} is a bad member of a group who causes problems for the rest of the group."],["vis",[{"t":"A few {it}bad apples{\\/it} cheated on the test, and now everyone has to take the test again."},{"t":"One {it}rotten apple{\\/it} ruined the day for the rest of us."}]]]],["text","{dx}see also {dxt|adam\\u0027s apple||} {dxt|crab apple||}{\\/dx}"]]}]]]}],"dros":[{"drp":"compare apples to apples\\/oranges","def":[{"sseq":[[["sense",{"dt":[["text","{dx}see {dxt|compare:1||}{\\/dx}"]]}]]]}]},{"drp":"the apple of someone\\u0027s eye","def":[{"sseq":[[["sense",{"dt":[["text","{bc}a person or thing that someone loves very much "],["vis",[{"t":"His daughter is {it}the apple of his eye{\\/it}."}]]]}]]]}]},{"drp":"upset the apple cart","def":[{"sseq":[[["sense",{"dt":[["text","{dx}see {dxt|upset:2||}{\\/dx}"]]}]]]}]}],"shortdef":["a round fruit with red, yellow, or green skin and firm white flesh \\u2014often used before another noun"]},{"meta":{"id":"apple-cheeked","uuid":"bcd1eaf2-735f-47d1-abee-fb9240ae9937","src":"learners","section":"alpha","target":{"tuuid":"e4de0b04-6301-4403-a4d2-84dc8f9f9d8f","tsrc":"collegiate"},"stems":["apple-cheeked"],"app-shortdef":{"hw":"apple-cheeked","fl":"adjective","def":["{bc} having red or pink cheeks"]},"offensive":false},"hwi":{"hw":"apple-cheeked","prs":[{"ipa":"\\u02c8\\u00e6p\\u0259l\\u02cct\\u0283i\\u02d0kt"}]},"fl":"adjective","def":[{"sseq":[[["sense",{"dt":[["text","{bc}having red or pink cheeks "],["vis",[{"t":"{it}apple-cheeked{\\/it} youngsters"}]]]}]]]}],"shortdef":["having red or pink cheeks"]},{"meta":{"id":"apple pie","uuid":"581b7222-3531-4cea-b167-bcd7b5846e41","src":"learners","section":"alpha","stems":["apple pie","apple pies","(as) american as apple pie","american as apple pie","as american as apple pie","in apple-pie order"],"app-shortdef":{"hw":"apple pie","fl":"noun","def":["{bc} a sweet pie made with apples"]},"offensive":false},"hwi":{"hw":"apple pie"},"fl":"noun","ins":[{"il":"plural","ifc":"~ pies","if":"apple pies"}],"gram":"count","def":[{"sseq":[[["sense",{"dt":[["text","{bc}a sweet pie made with apples"]]}]]]}],"dros":[{"drp":"(as) American as apple pie","def":[{"sseq":[[["sense",{"dt":[["text","{bc}very or typically American "],["vis",[{"t":"Baseball is {it}as American as apple pie{\\/it}."}]]]}]]]}]},{"drp":"in apple-pie order","def":[{"sseq":[[["sense",{"sls":["informal"],"dt":[["text","{bc}arranged neatly or perfectly {bc}in perfect order "],["vis",[{"t":"Everything in the cupboard was (arranged) {it}in apple-pie order{\\/it}."}]]]}]]]}]}],"shortdef":["a sweet pie made with apples"]},{"meta":{"id":"apple polisher","uuid":"ffe24101-8aee-461a-aaa9-5c981a764427","src":"learners","section":"alpha","stems":["apple polisher","apple polishers"],"app-shortdef":{"hw":"apple polisher","fl":"noun","def":["{it}US{\\/it}, {it}informal + disapproving{\\/it} {bc} a person who tries to get the approval and friendship of someone in authority by praise, flattery, etc."]},"offensive":false},"hwi":{"hw":"apple polisher"},"fl":"noun","ins":[{"il":"plural","ifc":"~ -ers","if":"apple polishers"}],"gram":"count","def":[{"sseq":[[["sense",{"sls":["US","informal + disapproving"],"dt":[["text","{bc}a person who tries to get the approval and friendship of someone in authority by praise, flattery, etc. "],["vis",[{"t":"an executive surrounded by {it}apple polishers{\\/it}"}]]]}]]]}],"shortdef":["a person who tries to get the approval and friendship of someone in authority by praise, flattery, etc."]},{"meta":{"id":"Adam\\u0027s apple","uuid":"eb16d36f-38e1-499d-8d0e-dc92397aa56c","src":"learners","section":"alpha","target":{"tuuid":"f7f0bd44-5d66-476b-abea-2cc3c3086b3b","tsrc":"collegiate"},"stems":["adam\\u0027s apple","adams apple","adam\\u0027s apples","adams apples"],"app-shortdef":{"hw":"Adam\\u0027s apple","fl":"noun","def":["{bc} the lump that sticks out in the front of a person\\u0027s neck, that is usually larger in men than in women, and that moves when a person talks or swallows"]},"offensive":false},"hwi":{"hw":"Adam\\u0027s apple"},"fl":"noun","ins":[{"il":"plural","ifc":"~ apples","if":"Adam\\u0027s apples"}],"gram":"count","def":[{"sseq":[[["sense",{"dt":[["text","{bc}the lump that sticks out in the front of a person\\u0027s neck, that is usually larger in men than in women, and that moves when a person talks or swallows"]]}]]]}],"shortdef":["the lump that sticks out in the front of a person\\u0027s neck, that is usually larger in men than in women, and that moves when a person talks or swallows"]},{"meta":{"id":"Big Apple","uuid":"6d00925e-431f-4ca1-8ac7-008092764310","src":"learners","section":"alpha","stems":["big apple","the big apple"],"app-shortdef":{"hw":"Big Apple","fl":"noun","def":["{it}informal{\\/it} used as a name for New York City"]},"offensive":false},"hwi":{"hw":"Big Apple"},"fl":"noun","dros":[{"drp":"the Big Apple","def":[{"sseq":[[["sense",{"sls":["informal"],"dt":[["uns",[[["text","used as a name for New York City "],["vis",[{"t":"She moved to {it}the Big Apple{\\/it} after she graduated."}]]]]]]}]]]}]}],"shortdef":[]},{"meta":{"id":"candy apple","uuid":"60ef9529-6409-413f-93d0-685b4df351ec","src":"learners","section":"alpha","stems":["candy apple","candy apples"],"app-shortdef":{"hw":"candy apple","fl":"noun","def":["{it}US{\\/it} {bc} an apple that is covered with a sugary mixture that becomes hard"]},"offensive":false},"hwi":{"hw":"candy apple"},"fl":"noun","ins":[{"il":"plural","ifc":"~ apples","if":"candy apples"}],"gram":"count","def":[{"sseq":[[["sense",{"sls":["US"],"dt":[["text","{bc}an apple that is covered with a sugary mixture that becomes hard"]]}]]]}],"shortdef":["an apple that is covered with a sugary mixture that becomes hard"]},{"meta":{"id":"crab apple","uuid":"aaf6f331-33e0-4f6e-bb71-950bf127faa1","src":"learners","section":"alpha","target":{"tuuid":"a77653b6-6d24-4a83-8142-40763b24460a","tsrc":"collegiate"},"stems":["crab apple","crab apples"],"app-shortdef":{"hw":"crab apple","fl":"noun","def":["{bc} a small, sour apple or the kind of tree that produces it"]},"offensive":false},"hwi":{"hw":"crab apple"},"fl":"noun","ins":[{"il":"plural","ifc":"~ apples","if":"crab apples"}],"gram":"count","def":[{"sseq":[[["sense",{"dt":[["text","{bc}a small, sour apple or the kind of tree that produces it"]]}]]]}],"shortdef":["a small, sour apple or the kind of tree that produces it"]},{"meta":{"id":"American:2","uuid":"f0722e99-1136-42da-951c-220aaede1b27","src":"learners","section":"alpha","target":{"tuuid":"296ccd06-4b79-4f57-869b-c7be89668420","tsrc":"collegiate"},"stems":["american","americanness","americannesses","(as) american as apple pie","american as apple pie","as american as apple pie"],"app-shortdef":{"hw":"American:2","fl":"adjective","def":["{bc} of or relating to the U.S. or its citizens","{bc} of or relating to North America, South America, or the people who live there"]},"offensive":false},"hom":2,"hwi":{"hw":"American","altprs":[{"ipa":"\\u0259\\u02c8mer\\u0259k\\u0259n"}]},"fl":"adjective","def":[{"sseq":[[["sense",{"sn":"1","dt":[["text","{bc}of or relating to the U.S. or its citizens "],["vis",[{"t":"{it}American{\\/it} culture\\/government\\/history"},{"t":"the {it}American{\\/it} people"},{"t":"their {it}American{\\/it} friends"}]]]}]],[["sense",{"sn":"2","dt":[["text","{bc}of or relating to North America, South America, or the people who live there "],["vis",[{"t":"the {it}American{\\/it} continents"},{"t":"a tropical {it}American{\\/it} tree"}]]]}]]]}],"dros":[{"drp":"(as) American as apple pie","def":[{"sseq":[[["sense",{"dt":[["text","{dx}see {dxt|apple pie||}{\\/dx}"]]}]]]}]}],"shortdef":["of or relating to the U.S. or its citizens","of or relating to North America, South America, or the people who live there"]},{"meta":{"id":"bad:1","uuid":"4f14c84e-d024-4da6-8802-dde4a1948b56","src":"learners","section":"alpha","target":{"tuuid":"a0b061c5-0990-4f66-874a-dd475a78dce5","tsrc":"collegiate"},"highlight":"yes","stems":["bad","badder","baddest","badness","badnesses","not bad","worse","worser","worst","a bad job","a bad lot","bad apple","come to a bad end","from bad to worse","in a bad way","in someone\\u0027s bad books","in someones bad books","too bad","with bad grace","bad debt","bad loan","bad cholesterol","bad guys","feel bad","gone bad"],"app-shortdef":{"hw":"bad:1","fl":"adjective","def":["{bc} low or poor in quality","{bc} not correct or proper","{bc} not pleasant, pleasing, or enjoyable"]},"offensive":false},"hom":1,"hwi":{"hw":"bad","prs":[{"ipa":"\\u02c8b\\u00e6d","sound":{"audio":"bad00001"}}]},"fl":"adjective","ins":[{"if":"worse","prs":[{"ipa":"\\u02c8w\\u025as","sound":{"audio":"worse001"}}]},{"if":"worst","prs":[{"ipa":"\\u02c8w\\u025ast","sound":{"audio":"worst001"}}]}],"def":[{"sseq":[[["sense",{"sn":"1 a","dt":[["text","{bc}low or poor in quality "],["vis",[{"t":"a {it}bad{\\/it} repair job"},{"t":"{it}bad{\\/it} work"},{"t":"The house is in {it}bad{\\/it} condition\\/shape."}]]]}],["sense",{"sn":"b","dt":[["text","{bc}not correct or proper "],["vis",[{"t":"{it}bad{\\/it} manners"},{"t":"{it}bad{\\/it} [={it}incorrect, faulty{\\/it}] grammar"},{"t":"a letter written in {it}bad{\\/it} French"},{"t":"{it}bad{\\/it} spelling"},{"t":"a {it}bad{\\/it} check [=a check that cannot be cashed]"}]],["text","{dx}see also {dxt|bad language||}{\\/dx}"]]}]],[["sense",{"sn":"2 a","dt":[["text","{bc}not pleasant, pleasing, or enjoyable "],["vis",[{"t":"He had a {it}bad{\\/it} day at the office."},{"t":"I was having a {it}bad{\\/it} dream."},{"t":"She made a very {it}bad{\\/it} impression on her future colleagues."},{"t":"The food tastes {it}bad{\\/it}."},{"t":"The flower smells {it}bad{\\/it}."},{"t":"He has {it}bad{\\/it} breath. [=breath that smells bad]"},{"t":"We\\u0027ve been having {it}bad{\\/it} weather lately."},{"t":"The medicine left a {it}bad{\\/it} taste in his mouth."},{"t":"It feels {it}bad{\\/it} [={it}uncomfortable, painful{\\/it}] to stretch out my arm."},{"t":"I look {it}bad{\\/it} in this hat. = This hat looks {it}bad{\\/it} on me."},{"t":"That hat doesn\\u0027t look {it}bad{\\/it} on you. [=that hat looks good on you]"}]]]}],["sense",{"sn":"b","dt":[["text","{bc}having, marked by, or relating to problems, troubles, etc. "],["vis",[{"t":"good and {it}bad{\\/it} news"},{"t":"They have remained together in good times and {it}bad{\\/it} (times)."},{"t":"It\\u0027s a {it}bad{\\/it} time for business right now."},{"t":"a {it}bad{\\/it} omen"},{"t":"{it}bad{\\/it} luck\\/fortune"},{"t":"Things are looking pretty {it}bad{\\/it} for us at this point."},{"t":"I have a {it}bad{\\/it} feeling about this."}]],["text","{dx}see also {dxt|bad blood||} {dxt|bad news||}{\\/dx}"]]}],["sense",{"sn":"c","dt":[["text","{bc}not adequate or suitable "],["vis",[{"t":"I couldn\\u0027t take a picture because the lighting was {it}bad{\\/it}."},{"t":"It\\u0027s a {it}bad{\\/it} day for a picnic."},{"t":"She made a {it}bad{\\/it} marriage."},{"t":"Is this a {it}bad{\\/it} moment\\/time to have a word with you?"}]]]}],["sense",{"sn":"d","dt":[["text","{bc}not producing or likely to produce a good result "],["vis",[{"t":"a {it}bad{\\/it} deal"},{"t":"a {it}bad{\\/it} risk"},{"t":"a {it}bad{\\/it} idea\\/plan"},{"t":"The plan has its good points and its {it}bad{\\/it} points."},{"t":"a {phrase}bad debt{\\/phrase} [=a debt that will not be paid]"},{"t":"a {phrase}bad loan{\\/phrase} [=a loan that will not be repaid]"}]]]}],["sense",{"sn":"e","dt":[["text","{bc}expressing criticism or disapproval "],["vis",[{"t":"The movie got {it}bad{\\/it} reviews."}]]]}]],[["sense",{"sn":"3 a","dt":[["text","{bc}not healthy {bc}marked or affected by injury or disease "],["vis",[{"t":"His health is pretty {it}bad{\\/it}. = He\\u0027s in pretty {it}bad{\\/it} health."},{"t":"The patient was pretty {it}bad{\\/it} [={it}ill, sick{\\/it}] last week and even {it}worse{\\/it} yesterday but is doing better now."},{"t":"He came home early because he was feeling pretty {it}bad{\\/it}. [=he wasn\\u0027t feeling well]"},{"t":"My father has a {it}bad{\\/it} back\\/leg. [=a back\\/leg that is always or often painful]"},{"t":"She has {it}bad{\\/it} eyesight\\/hearing."},{"t":"{it}bad{\\/it} teeth"}]]]}],["sense",{"sn":"b","dt":[["text","{bc}causing harm or trouble "],["vis",[{"t":"a {it}bad{\\/it} diet"},{"t":"a {it}bad{\\/it} influence"},{"t":"{phrase}bad cholesterol{\\/phrase} [=a type of cholesterol that can cause serious health problems when there is too much of it in your blood]"}]],["uns",[[["text","often + {it}for{\\/it} "],["vis",[{"t":"Eating too much can be {it}bad for{\\/it} you. = It can be {it}bad for{\\/it} you to eat too much."},{"t":"Eating all that candy is {it}bad for{\\/it} your teeth."},{"t":"Watching too much TV is {it}bad for{\\/it} children."}]]]]]]}]],[["sense",{"sn":"4 a","dt":[["text","{bc}not morally good or right {bc}morally evil or wrong "],["vis",[{"t":"a {it}bad{\\/it} person"},{"t":"{it}bad{\\/it} conduct\\/behavior"},{"t":"a man of {it}bad{\\/it} character"},{"t":"{it}bad{\\/it} intentions\\/deeds"},{"t":"It\\u0027s hard to tell the good guys from the {phrase}bad guys{\\/phrase} in this movie."}]],["text","{dx}see also {dxt|bad faith||}{\\/dx}"]]}],["sense",{"sn":"b","dt":[["text","{bc}not behaving properly "],["vis",[{"t":"a {it}bad{\\/it} dog"},{"t":"I\\u0027m afraid your son has been a very {it}bad{\\/it} [={it}naughty{\\/it}] boy."}]],["text","{dx}see also {dxt|bad boy||}{\\/dx}"]]}]],[["sense",{"sn":"5 a","dt":[["text","{bc}not skillful {bc}not doing or able to do something well "],["vis",[{"t":"a {it}bad{\\/it} musician"},{"t":"a {it}bad{\\/it} doctor"},{"t":"She was pretty {it}bad{\\/it} in that movie. [=she did not act well]"}]],["uns",[[["text","often + {it}at{\\/it} "],["vis",[{"t":"a doctor who\\u0027s {it}bad at{\\/it} treating nervous patients"},{"t":"He\\u0027s very\\/really {it}bad at{\\/it} expressing his true feelings."}]]]]]]}],["sense",{"sn":"b","dt":[["text","{bc}having a tendency not to do something"],["uns",[[["text","+ {it}about{\\/it} "],["vis",[{"t":"He\\u0027s {it}bad about{\\/it} getting to work on time. [=he often fails to get to work on time]"},{"t":"I\\u0027m very {it}bad about{\\/it} remembering people\\u0027s birthdays. [=I often forget people\\u0027s birthdays]"}]]]]]]}]],[["sense",{"sn":"6 a","dt":[["text","{bc}not happy or pleased {bc}feeling regret or guilt about something "],["vis",[{"t":"I {phrase}feel bad{\\/phrase} about what happened. [=I regret what happened]"},{"t":"She {it}felt bad{\\/it} that she forgot to call. = She {it}felt bad{\\/it} about forgetting to call."}]]]}],["sense",{"sn":"b","dt":[["text","{bc}not cheerful or calm "],["vis",[{"t":"She\\u0027s in a {it}bad{\\/it} mood. [=an angry mood]"},{"t":"He has a {it}bad{\\/it} temper. [=he\\u0027s bad-tempered; he becomes angry easily]"}]]]}]],[["sense",{"sn":"7","dt":[["text","{bc}serious or severe "],["vis",[{"t":"She\\u0027s in {it}bad{\\/it} trouble."},{"t":"He has a {it}bad{\\/it} cough\\/cold."},{"t":"That bruise looks {it}bad{\\/it}: you\\u0027d better see a doctor about it."},{"t":"How {it}bad{\\/it} is the pain?"}]]]}]],[["sense",{"sn":"8","dt":[["text","{bc}no longer good to eat or drink {bc}not fresh "],["vis",[{"t":"the smell of {it}bad{\\/it} fish"},{"t":"Is the milk still good or has it {phrase}gone bad{\\/phrase}? [={it}spoiled{\\/it}]"}]]]}]],[["sen",{"sn":"9","ins":[{"if":"bad*der"},{"if":"bad*dest"}],"sls":["chiefly US","informal"]}],["sense",{"sn":"a","dt":[["text","{bc}very good "],["vis",[{"t":"He\\u0027s the {it}baddest{\\/it} guitar player you\\u0027ll ever hear!"}]]]}],["sense",{"sn":"b","dt":[["text","{bc}very tough or dangerous "],["vis",[{"t":"Don\\u0027t mess around with him. He\\u0027s a {it}bad{\\/it} dude."}]]]}]]]}],"uros":[{"ure":"bad*ness","fl":"noun","gram":"noncount","utxt":[["vis",[{"t":"the {it}badness{\\/it} of his behavior"},{"t":"the {it}badness{\\/it} [={it}severity{\\/it}] of his injuries"},{"t":"There\\u0027s more goodness than {it}badness{\\/it} in him."}]]]}],"dros":[{"drp":"a bad job","def":[{"sseq":[[["sense",{"dt":[["text","{dx}see {dxt|job||}{\\/dx}"]]}]]]}]},{"drp":"a bad lot","def":[{"sseq":[[["sense",{"dt":[["text","{dx}see {dxt|lot||}{\\/dx}"]]}]]]}]},{"drp":"bad apple","def":[{"sseq":[[["sense",{"dt":[["text","{dx}see {dxt|apple||}{\\/dx}"]]}]]]}]},{"drp":"come to a bad end","def":[{"sseq":[[["sense",{"dt":[["text","{dx}see {dxt|end:1||}{\\/dx}"]]}]]]}]},{"drp":"from bad to worse","def":[{"sseq":[[["sense",{"dt":[["text","{bc}from a bad state or condition to an even worse state or condition "],["vis",[{"t":"The company has been struggling for years, and things have recently gone {it}from bad to worse{\\/it}."}]]]}]]]}]},{"drp":"in a bad way","def":[{"sseq":[[["sense",{"dt":[["text","{bc}in a bad condition "],["vis",[{"t":"Without enough funding, public services are {it}in a{\\/it} pretty {it}bad way{\\/it} right now."},{"t":"The patient was {it}in a bad way{\\/it} last week but is doing better now."}]]]}]]]}]},{"drp":"in someone\\u0027s bad books","def":[{"sseq":[[["sense",{"dt":[["text","{dx}see {dxt|book:1||}{\\/dx}"]]}]]]}]},{"drp":"not bad","def":[{"sseq":[[["sense",{"dt":[["text","{bc}fairly good or quite good "],["vis",[{"t":"All things considered, she\\u0027s {it}not{\\/it} a {it}bad{\\/it} singer. [=she\\u0027s a pretty good singer]"},{"t":"{ldquo}How are you?{rdquo} {ldquo}{it}Not{\\/it} (too\\/so) {it}bad{\\/it}, thanks. And you?{rdquo}"}]]]}]]]}]},{"drp":"too bad","def":[{"sseq":[[["sense",{"sn":"1","dt":[["uns",[[["text","used to show that you are sorry or feel bad about something "],["vis",[{"t":"It\\u0027s {it}too bad{\\/it} [={it}unfortunate{\\/it}] that John and Mary are getting divorced. = It\\u0027s {it}too bad{\\/it} about John and Mary getting divorced. [=I\\u0027m sorry to hear that John and Mary are getting divorced]"},{"t":"{ldquo}I won\\u0027t be able to come to the party.{rdquo} {ldquo}(That\\u0027s) {it}Too bad{\\/it}. I was hoping you\\u0027d be there.{rdquo}"}]]]]]]}]],[["sense",{"sn":"2","dt":[["uns",[[["text","used in an ironic way to show that you are not sorry or do not feel bad about something "],["vis",[{"t":"{ldquo}But I need your help!{rdquo} {ldquo}(That\\u0027s just) {it}Too bad{\\/it}.{rdquo}"}]]]]]]}]]]}]},{"drp":"with bad grace","def":[{"sseq":[[["sense",{"dt":[["text","{dx}see {dxt|grace:1||}{\\/dx}"]]}]]]}]}],"shortdef":["low or poor in quality","not correct or proper","not pleasant, pleasing, or enjoyable"]}]

In [ ]:
dicX

In [ ]:
wordin_st(st,'zannenbaum')

In [ ]:
head_tail_st(st)

####  Submit Employee Attrition Webservice Requests ( * open * )

In [ ]:
today = date.today()
e = datetime.datetime.now()

parsed = df.to_dict(orient='index')
dfo = pd.DataFrame()

# print(parsed)

for k,v in parsed.items():
        r = empatt(v)
        r = json.loads(r)
        r = r['Results'][serviceoutput][0]
        dftemp = pd.DataFrame.from_dict(r,orient ='index').transpose()
        dfo = dfo.append(dftemp,ignore_index=True)
        suc = 'Match' if dftemp.loc[0,'Attrition']==dftemp.loc[0,'Scored Labels'] else '* Mismatch *'
        print(f'Processing record = {k+1:3} of {len(parsed):3} {suc:10}')
    
file = filepath + 'Emp_Attrition (WebService Output)' + ext_c
dfo.to_csv(file,index=False)